In [1]:
!pip install langchain_community
!pip install openai
!pip install IPython
!pip install pypdf
!pip install chromadb
!pip install -U langchain-openai
!pip install unstructured[all-docs]
!apt-get install -y poppler-utils
!apt-get install -y poppler-utils
!pip install unstructured[all-docs]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.5 MB/s eta 0:00:00

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://security.ubuntu.com/ubuntu jammy-security/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.10 [186 kB]
Fetched 186 kB in 0s (445 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126371 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.10_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.10) ...
Setting up poppler-utils (22.02.0-2ubuntu0.10) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.10).
0 upgraded, 0 newly

In [2]:
import os
from openai import OpenAI
import json
import requests
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from glob import glob
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.document_loaders import UnstructuredPDFLoader
from glob import glob

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
file_name = '/content/drive/MyDrive/RAG Programs/RAG Coding/RAG Multiple Files/config.json'

with open(file_name, 'r') as file:
  config = json.load(file)
  os.environ['OPENAI_API_KEY'] = config.get('OPENAI_API_KEY')

In [ ]:
#@title Load the PDF file

CHROMA_PATH = "ai-overview-db"

DOC_FOLDER = "/content/drive/MyDrive/RAG Programs/RAG Coding/RAG Multiple Files/pdfs/"

pdf_files = glob(f"{DOC_FOLDER}*.pdf")

all_documents = []
for pdf_path in pdf_files:
    # Use UnstructuredPDFLoader for better parsing, especially for complex layouts
    loader = UnstructuredPDFLoader(pdf_path, mode="elements", strategy="hi_res")
    documents = loader.load()
    all_documents.extend(documents)

for doc in all_documents:
  doc.metadata = filter_complex_metadata(doc.metadata)

yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    separators=["\n\n", "\n", " ", ""]

chunks = text_splitter.split_documents(all_documents)

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=config['OPENAI_API_KEY'])

# Create a new ChromaDB instance from the chunks
db_chroma = Chroma.from_documents(chunks, embeddings)

# Create a re-ranking model
compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_retriever=db_chroma.as_retriever(search_kwargs={"k": 20}),
    base_compressor=compressor,
)

ValueError: Expected metadata value to be a str, int, float, bool, or None, got {'points': ((np.float64(69.86636055555554), np.float64(69.78759750000008)), (np.float64(69.86636055555554), np.float64(1288.2399608333333)), (np.float64(1282.6274713888888), np.float64(1288.2399608333333)), (np.float64(1282.6274713888888), np.float64(69.78759750000008))), 'system': 'PixelSpace', 'layout_width': 2667, 'layout_height': 1500} which is a dict in upsert.

Try filtering complex metadata from the document using langchain_community.vectorstores.utils.filter_complex_metadata.

In [ ]:
query = "What should we fear about AI?"
relevant_docs = compression_retriever.get_relevant_documents(query)

context_text = "\n\n---\n\n".join([d.page_content for d in relevant_docs])

In [ ]:
PROMPT_TEMPLATE = """

Answer the question based only on the follow context:

{context}

Answer the question based on the above context: {question}.

Provide a detailed answer.
Don't justify your answers.
Don't give information not mentioned in the CONTEXT INFORMATION.
Do not say "according to the context" or "mentioned in the context" or anything similar.

"""

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question = query)
print(prompt)